# 01 - CNN preprocessing comparison

__Andrés Felipe García Albarracín <br>
May 7, 2021__

In [10]:
import tensorflow as tf
import os
from shutil import copyfile
import pandas as pd
import time

## 1. Order data in the training / validation folders

In [2]:
# Run the following the first time

trainingFolder = "../Datasets/flowers/training"
validationFolder = "../Datasets/flowers/validation"
firstTime = not(os.path.exists(trainingFolder) and os.path.exists(validationFolder))

def safeCreateFolder(folderPath):
    """
    Creates a folder if it does not exist
    """
    if not os.path.exists(folderPath):
        os.mkdir(folderPath)

if firstTime:
    # Unzip files
    from zipfile import ZipFile

    with ZipFile(os.path.join("../Datasets/archive.zip"), 'r') as zipObj:
        zipObj.extractall(os.path.join("../Datasets/flowers"))

    # Create folders
    safeCreateFolder(trainingFolder)
    safeCreateFolder(validationFolder)
    
    # Split function
    def splitFiles(sourcePath, trainingPath, validationPath, splitSize):
        """
        Function that splits the files from sourcePath in trainingPath and
        validationPath, according to the splitSize
        """
        folderList = os.listdir(sourcePath)
        for folder in folderList:
            safeCreateFolder(os.path.join(trainingPath,folder))
            safeCreateFolder(os.path.join(validationPath,folder))
            fileList = os.listdir(os.path.join(sourcePath, folder))
            for n, file in enumerate(fileList):
                if n < splitSize * len(fileList):
                    copyfile(
                        os.path.join(sourcePath, folder, file),
                        os.path.join(trainingPath, folder, file)
                    )
                else:
                    copyfile(
                        os.path.join(sourcePath, folder, file),
                        os.path.join(validationPath, folder, file)
                )
    
    # Call the functions
    splitRatio = 0.7
    splitFiles(
        sourcePath = "../Datasets/flowers/flowers",
        trainingPath = trainingFolder,
        validationPath = validationFolder,
        splitSize = splitRatio
    )

## 2. Define the model to use

In [3]:
def createAndCompile(numClases):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation = 'relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(numClases, activation = 'softmax')
    ])
    model.compile(
        optimizer = tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.CategoricalCrossentropy(),
        metrics = ['accuracy']
    )
    return model


In [5]:
numClases = len(os.listdir(trainingFolder))

## 3. Strategies

In [7]:
dfResults = pd.DataFrame(columns=['Experiment', 'Training time', 'Evaluation time'])

### 3.1 Image Data Generator

In [4]:
trainDataGen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
train_generator = trainDataGen.flow_from_directory(
    trainingFolder,
    batch_size = 256,
    target_size = (150,150),
    class_mode = 'categorical'
)


Found 3028 images belonging to 5 classes.


In [13]:
start = time.time()
model1 = createAndCompile(numClases)
model1.fit(
    train_generator,
    epochs = 10,
    steps_per_epoch = 12
)
totalTime = time.time() - start

Epoch 1/10
12/12 [==============================] - 82s 7s/step - loss: 5.5560 - accuracy: 0.2454
Epoch 2/10
12/12 [==============================] - 87s 7s/step - loss: 1.3886 - accuracy: 0.3699
Epoch 3/10
12/12 [==============================] - 90s 7s/step - loss: 1.1822 - accuracy: 0.5234
Epoch 4/10
12/12 [==============================] - 93s 8s/step - loss: 1.0164 - accuracy: 0.5978
Epoch 5/10
10/12 [========================>.....] - ETA: 14s - loss: 0.8326 - accuracy: 0.6860

KeyboardInterrupt: 

In [11]:
dfResults

11.828125